In [33]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib backend BEFORE importing pyplot
import os
os.environ['MPLBACKEND'] = 'Agg'

import matplotlib.pyplot as plt
import seaborn as sns

# Deep Learning imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Sklearn imports
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print("Environment setup complete.")

TensorFlow version: 2.20.0
GPU available: False
Environment setup complete.


In [34]:
# Define paths
BASE_DIR = Path(r'c:\Users\miray\Desktop\dengue_forecasting_project')
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'
MODELS_DIR = BASE_DIR / 'models'
MODELS_DIR.mkdir(exist_ok=True)

# Load engineered data
data_file = PROCESSED_DIR / 'feature_engineered_data.csv'
df = pd.read_csv(data_file, parse_dates=['Date'], index_col='Date')

print(f"Loaded data: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Total weeks: {len(df)}")

Loaded data: (318, 325)
Date range: 2016-02-05 00:00:00 to 2022-09-03 00:00:00
Total weeks: 318


In [35]:
# Separate features and targets
target_cols = [col for col in df.columns if '_target' in col]
feature_cols = [col for col in df.columns if '_target' not in col]

print(f"Original features: {len(feature_cols)}")
print(f"Targets: {len(target_cols)}")

# Select MOST important features only (aggressive reduction for small dataset)
important_features = []

# 1. Current dengue cases (most important)
dengue_current = [col for col in feature_cols if 'Cases' in col and 'lag' not in col and 'roll' not in col]
important_features.extend(dengue_current)
print(f"\n1. Current dengue cases: {len(dengue_current)}")

# 2. Dengue lag features (t-1, t-2 only - reduced from 4 lags)
dengue_lags = [col for col in feature_cols if 'Cases' in col and any(f'lag{i}' in col for i in [1,2])]
important_features.extend(dengue_lags)
print(f"2. Dengue lag features: {len(dengue_lags)}")

# 3. Dengue rolling mean (3-week only)
dengue_rolling = [col for col in feature_cols if 'Cases' in col and 'roll3' in col]
important_features.extend(dengue_rolling)
print(f"3. Dengue rolling means: {len(dengue_rolling)}")

# 4. Key weather (current only - no lags)
weather_key = [col for col in feature_cols if 
               (('Rainfall' in col or 'Temp_Avg' in col) and 
                'lag' not in col and 'roll' not in col)]
important_features.extend(weather_key)
print(f"4. Key weather features: {len(weather_key)}")

# 5. Essential temporal only (month, sin/cos encodings)
temporal = [col for col in feature_cols if any(x in col for x in ['month', 'sin', 'cos'])]
important_features.extend(temporal)
print(f"5. Temporal features: {len(temporal)}")

# Remove duplicates
important_features = list(dict.fromkeys(important_features))

print(f"\n✅ Selected {len(important_features)} important features (reduced from {len(feature_cols)})")
print(f"Feature reduction: {len(feature_cols)} → {len(important_features)} ({len(important_features)/len(feature_cols)*100:.1f}%)")

Original features: 317
Targets: 8

1. Current dengue cases: 8
2. Dengue lag features: 16
3. Dengue rolling means: 8
4. Key weather features: 16
5. Temporal features: 5

✅ Selected 53 important features (reduced from 317)
Feature reduction: 317 → 53 (16.7%)


In [36]:
# Create reduced dataset
X = df[important_features].values
y = df[target_cols].values
dates = df.index.values

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nFeature-to-sample ratio: {X.shape[1]}/{X.shape[0]} = {X.shape[1]/X.shape[0]:.3f}")
print("(Lower is better - ideal < 0.1)")

X shape: (318, 53)
y shape: (318, 8)

Feature-to-sample ratio: 53/318 = 0.167
(Lower is better - ideal < 0.1)


In [37]:
def create_sequences(X, y, dates, lookback=4):
    """
    Create sequences for LSTM training
    """
    X_seq, y_seq, dates_seq = [], [], []
    
    for i in range(lookback, len(X)):
        X_seq.append(X[i-lookback:i])
        y_seq.append(y[i])
        dates_seq.append(dates[i])
    
    return np.array(X_seq), np.array(y_seq), np.array(dates_seq)

# Create sequences with 4-week lookback (dengue has ~4-week lag from weather)
LOOKBACK = 4
X_seq, y_seq, dates_seq = create_sequences(X, y, dates, lookback=LOOKBACK)

print(f"Sequence shapes:")
print(f"  X_seq: {X_seq.shape} (samples, lookback, features)")
print(f"  y_seq: {y_seq.shape} (samples, targets)")
print(f"\nData: {len(X)} → {len(X_seq)} samples (lost {len(X) - len(X_seq)} due to lookback)")

Sequence shapes:
  X_seq: (314, 4, 53) (samples, lookback, features)
  y_seq: (314, 8) (samples, targets)

Data: 318 → 314 samples (lost 4 due to lookback)


In [38]:
# Split ratios - MAXIMIZED training data for small dataset (80/10/10)
train_ratio = 0.80
val_ratio = 0.10
test_ratio = 0.10

n_samples = len(X_seq)
train_end = int(n_samples * train_ratio)
val_end = int(n_samples * (train_ratio + val_ratio))

# Split data
X_train, y_train = X_seq[:train_end], y_seq[:train_end]
X_val, y_val = X_seq[train_end:val_end], y_seq[train_end:val_end]
X_test, y_test = X_seq[val_end:], y_seq[val_end:]

dates_train = dates_seq[:train_end]
dates_val = dates_seq[train_end:val_end]
dates_test = dates_seq[val_end:]

print(f"Data split:")
print(f"  Train: {X_train.shape[0]} samples ({train_ratio*100:.0f}%)")
print(f"    Date range: {pd.Timestamp(dates_train[0]).strftime('%Y-%m-%d')} to {pd.Timestamp(dates_train[-1]).strftime('%Y-%m-%d')}")
print(f"  Val:   {X_val.shape[0]} samples ({val_ratio*100:.0f}%)")
print(f"    Date range: {pd.Timestamp(dates_val[0]).strftime('%Y-%m-%d')} to {pd.Timestamp(dates_val[-1]).strftime('%Y-%m-%d')}")
print(f"  Test:  {X_test.shape[0]} samples ({test_ratio*100:.0f}%)")
print(f"    Date range: {pd.Timestamp(dates_test[0]).strftime('%Y-%m-%d')} to {pd.Timestamp(dates_test[-1]).strftime('%Y-%m-%d')}")

Data split:
  Train: 251 samples (80%)
    Date range: 2016-03-04 to 2021-01-08
  Val:   31 samples (10%)
    Date range: 2021-01-15 to 2021-08-13
  Test:  32 samples (10%)
    Date range: 2021-08-20 to 2022-09-03


In [39]:
# Reshape for scaling
n_samples_train, lookback, n_features = X_train.shape
X_train_flat = X_train.reshape(-1, n_features)
X_val_flat = X_val.reshape(-1, n_features)
X_test_flat = X_test.reshape(-1, n_features)

# Fit scaler on training data only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val_flat).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test_flat).reshape(X_test.shape)

print(f"Feature normalization complete:")
print(f"  Train: {X_train_scaled.shape}")
print(f"  Val:   {X_val_scaled.shape}")
print(f"  Test:  {X_test_scaled.shape}")

# Save scaler
import joblib
scaler_file = MODELS_DIR / 'feature_scaler_v2.pkl'
joblib.dump(scaler, scaler_file)
print(f"\n✅ Saved scaler to: {scaler_file}")

Feature normalization complete:
  Train: (251, 4, 53)
  Val:   (31, 4, 53)
  Test:  (32, 4, 53)

✅ Saved scaler to: c:\Users\miray\Desktop\dengue_forecasting_project\models\feature_scaler_v2.pkl


In [40]:
def build_v2_lstm(input_shape, output_dim):
    """
    Build simplified LSTM for small dataset
    
    Args:
        input_shape: (lookback, features)
        output_dim: Number of output locations (8)
    
    Returns:
        Compiled Keras model
    """
    model = Sequential([
        # Small LSTM layer with strong regularization
        LSTM(32, return_sequences=False, input_shape=input_shape, 
             kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001),
             name='lstm_1'),
        Dropout(0.4),
        
        # Single dense layer
        Dense(32, activation='relu', kernel_regularizer=l2(0.001), name='dense_1'),
        Dropout(0.3),
        
        # Output layer
        Dense(output_dim, activation='linear', name='output')
    ])
    
    # Compile with slightly higher learning rate
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae', 'mse']
    )
    
    return model

# Build model
input_shape = (LOOKBACK, X_train_scaled.shape[2])
output_dim = y_train.shape[1]

model = build_v2_lstm(input_shape, output_dim)

print("V2 Model Architecture (Optimized for Small Dataset):")
model.summary()

print("\n🔧 All Optimizations Applied:")
print(f"  • LSTM units: 32 (minimized for small dataset)")
print(f"  • L2 regularization: 0.001 (strong)")
print(f"  • LSTM dropout: 0.4 (stronger)")
print(f"  • Dense dropout: 0.3 (strong)")
print(f"  • Lookback: 4 weeks (dengue has ~4-week lag)")
print(f"  • Split: 80/10/10 (maximized training)")
print(f"  • Architecture: 1 LSTM + 1 Dense (simplified)")
print(f"  • Features: ~50 (aggressive selection)")

V2 Model Architecture (Optimized for Small Dataset):


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 32)             │        11,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,328 (48.16 KB)

 Trainable params: 12,328 (48.16 KB)

 Non-trainable params: 0 (0.00 B)


🔧 All Optimizations Applied:
  • LSTM units: 32 (minimized for small dataset)
  • L2 regularization: 0.001 (strong)
  • LSTM dropout: 0.4 (stronger)
  • Dense dropout: 0.3 (strong)
  • Lookback: 4 weeks (dengue has ~4-week lag)
  • Split: 80/10/10 (maximized training)
  • Architecture: 1 LSTM + 1 Dense (simplified)
  • Features: ~50 (aggressive selection)


In [41]:
# Define callbacks - NO EARLY STOPPING
callbacks = [
    ModelCheckpoint(
        filepath=str(MODELS_DIR / 'best_model_v2.keras'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

print("Callbacks configured:")
print("  ✅ Model checkpoint (save best)")

Callbacks configured:
  ✅ Model checkpoint (save best)


In [42]:
# Train model
print("Starting training V2 model...\n")

history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=200,  # More epochs with increased patience
    batch_size=8,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training complete!")

Starting training V2 model...

Epoch 1/200
Epoch 1/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 26750.5503 - mae: 58.9981 - mse: 26750.4077
Epoch 1: val_loss improved from None to 24.77313, saving model to c:\Users\miray\Desktop\dengue_forecasting_project\models\best_model_v2.keras

Epoch 1: val_loss improved from None to 24.77313, saving model to c:\Users\miray\Desktop\dengue_forecasting_project\models\best_model_v2.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - loss: 26170.0488 - mae: 57.4109 - mse: 26169.9082 - val_loss: 24.7731 - val_mae: 3.3266 - val_mse: 24.6320
Epoch 2/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - loss: 26170.0488 - mae: 57.4109 - mse: 26169.9082 - val_loss: 24.7731 - val_mae: 3.3266 - val_mse: 24.6320
Epoch 2/200
25/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 26631.1915 - mae: 59.2059 - mse: 26631.0511
Epoch 2: val_loss improved from 24.77313 to 23.03372, saving model to c:\Users\miray\Desktop\dengue_forecasting_project\models\best_model_v2.keras

Epoc

In [43]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss (MSE)', fontsize=12)
axes[0].set_title('Model Loss (V2 - Reduced Regularization)', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history.history['mae'], label='Train MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Val MAE', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('MAE', fontsize=12)
axes[1].set_title('Mean Absolute Error (V2)', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(MODELS_DIR / 'training_history_v2.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Training history plot saved")


✅ Training history plot saved


In [44]:
# Evaluate on test set
print("Evaluating on test set...\n")

test_loss, test_mae, test_mse = model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"Test Set Performance (V2 Model):")
print(f"  Loss (MSE): {test_loss:.4f}")
print(f"  MAE: {test_mae:.4f}")
print(f"  RMSE: {np.sqrt(test_mse):.4f}")

# Make predictions
y_pred_train = model.predict(X_train_scaled, verbose=0)
y_pred_val = model.predict(X_val_scaled, verbose=0)
y_pred_test = model.predict(X_test_scaled, verbose=0)

print("\n✅ Predictions generated")

Evaluating on test set...

Test Set Performance (V2 Model):
  Loss (MSE): 1635.9514
  MAE: 22.1757
  RMSE: 40.4266
Test Set Performance (V2 Model):
  Loss (MSE): 1635.9514
  MAE: 22.1757
  RMSE: 40.4266

✅ Predictions generated

✅ Predictions generated


In [45]:
# Calculate per-location metrics
location_names = [
    'Aklan', 'Antique', 'Bacolodcity', 'Capiz',
    'Guimaras', 'Iloilo', 'Iloilocity', 'Negrosocc'
]

print("\nPer-Location Test Set Metrics (V2 Model - Reduced Regularization):")
print("="*90)
print(f"{'Location':<15} {'RMSE':<10} {'MAE':<10} {'R²':<10} {'Mean Actual':<12} {'Mean Pred':<12} {'Status'}")
print("="*90)

metrics_summary = []
for i, location in enumerate(location_names):
    y_true_loc = y_test[:, i]
    y_pred_loc = y_pred_test[:, i]
    
    rmse = np.sqrt(mean_squared_error(y_true_loc, y_pred_loc))
    mae = mean_absolute_error(y_true_loc, y_pred_loc)
    r2 = r2_score(y_true_loc, y_pred_loc)
    
    # Status indicator
    if r2 > 0.3:
        status = "✅ Good"
    elif r2 > 0:
        status = "⚠️ Fair"
    elif r2 > -0.5:
        status = "⚠️ Poor"
    else:
        status = "❌ Very Poor"
    
    print(f"{location:<15} {rmse:<10.2f} {mae:<10.2f} {r2:<10.3f} {y_true_loc.mean():<12.2f} {y_pred_loc.mean():<12.2f} {status}")
    
    metrics_summary.append({
        'Location': location,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    })

print("="*90)

# Calculate average metrics
metrics_df = pd.DataFrame(metrics_summary)
print(f"\nAverage Performance:")
print(f"  Avg RMSE: {metrics_df['RMSE'].mean():.2f}")
print(f"  Avg MAE: {metrics_df['MAE'].mean():.2f}")
print(f"  Avg R²: {metrics_df['R2'].mean():.3f}")
print(f"  Locations with R² > 0: {(metrics_df['R2'] > 0).sum()}/8")
print(f"  Locations with R² > 0.3: {(metrics_df['R2'] > 0.3).sum()}/8")

# Save metrics
metrics_df.to_csv(MODELS_DIR / 'test_metrics_v2.csv', index=False)
print(f"\n✅ Metrics saved to: {MODELS_DIR / 'test_metrics_v2.csv'}")


Per-Location Test Set Metrics (V2 Model - Reduced Regularization):
Location        RMSE       MAE        R²         Mean Actual  Mean Pred    Status
Aklan           22.27      15.79      -2.513     11.38        24.89        ❌ Very Poor
Antique         65.64      39.28      -0.271     52.59        21.35        ⚠️ Poor
Bacolodcity     21.39      13.86      -1.489     14.28        25.07        ❌ Very Poor
Capiz           17.05      11.70      -0.785     18.16        20.35        ❌ Very Poor
Guimaras        4.01       2.68       0.644      6.53         4.94         ✅ Good
Iloilo          47.93      31.52      0.072      54.38        60.49        ⚠️ Fair
Iloilocity      19.02      12.16      0.536      16.41        16.83        ✅ Good
Negrosocc       69.62      50.42      0.165      93.88        92.69        ⚠️ Fair

Average Performance:
  Avg RMSE: 33.37
  Avg MAE: 22.18
  Avg R²: -0.455
  Locations with R² > 0: 4/8
  Locations with R² > 0.3: 2/8

✅ Metrics saved to: c:\Users\miray\Deskto

In [50]:
# Diagnostic Analysis
print("DIAGNOSTIC ANALYSIS")
print("="*100)

# 1. Analyze case distributions
print("\n1. CASE DISTRIBUTION ANALYSIS:")
print("-"*100)
for i, location in enumerate(location_names):
    train_cases = y_train[:, i]
    test_cases = y_test[:, i]
    
    print(f"\n{location}:")
    print(f"  Train: mean={train_cases.mean():.1f}, std={train_cases.std():.1f}, min={train_cases.min():.0f}, max={train_cases.max():.0f}")
    print(f"  Test:  mean={test_cases.mean():.1f}, std={test_cases.std():.1f}, min={test_cases.min():.0f}, max={test_cases.max():.0f}")
    print(f"  Train samples: {len(train_cases)}, Test samples: {len(test_cases)}")

# 2. Identify performance patterns
print("\n\n2. PERFORMANCE PATTERNS:")
print("-"*100)
good_locations = metrics_df[metrics_df['R2'] > 0.3]['Location'].tolist()
fair_locations = metrics_df[(metrics_df['R2'] > 0) & (metrics_df['R2'] <= 0.3)]['Location'].tolist()
poor_locations = metrics_df[metrics_df['R2'] <= 0]['Location'].tolist()

print(f"\nGood Performance (R² > 0.3): {good_locations}")
print(f"Fair Performance (0 < R² ≤ 0.3): {fair_locations}")
print(f"Poor Performance (R² ≤ 0): {poor_locations}")

# 3. Analyze prediction ranges
print("\n\n3. PREDICTION RANGE ANALYSIS:")
print("-"*100)
print(f"{'Location':<15} {'Actual Range':<20} {'Pred Range':<20} {'Coverage':<15}")
print("-"*100)
for i, location in enumerate(location_names):
    actual_range = f"{y_test[:, i].min():.0f} - {y_test[:, i].max():.0f}"
    pred_range = f"{y_pred_test[:, i].min():.1f} - {y_pred_test[:, i].max():.1f}"
    
    # Calculate how much of actual range is covered by predictions
    actual_span = y_test[:, i].max() - y_test[:, i].min()
    pred_span = y_pred_test[:, i].max() - y_pred_test[:, i].min()
    coverage = (pred_span / actual_span * 100) if actual_span > 0 else 0
    
    print(f"{location:<15} {actual_range:<20} {pred_range:<20} {coverage:<15.1f}%")

# 4. Check for overfitting
print("\n\n4. OVERFITTING CHECK:")
print("-"*100)
train_loss_final = history.history['loss'][-1]
val_loss_final = history.history['val_loss'][-1]
overfit_ratio = val_loss_final / train_loss_final

print(f"Final Training Loss: {train_loss_final:.4f}")
print(f"Final Validation Loss: {val_loss_final:.4f}")
print(f"Val/Train Ratio: {overfit_ratio:.2f}x")
if overfit_ratio > 2.0:
    print("⚠️ WARNING: Significant overfitting detected!")
elif overfit_ratio > 1.3:
    print("⚠️ Moderate overfitting")
else:
    print("✅ Overfitting under control")

print("\n" + "="*100)

DIAGNOSTIC ANALYSIS

1. CASE DISTRIBUTION ANALYSIS:
----------------------------------------------------------------------------------------------------

Aklan:
  Train: mean=42.6, std=69.7, min=0, max=473
  Test:  mean=11.4, std=11.9, min=0, max=44
  Train samples: 251, Test samples: 32

Antique:
  Train: mean=39.8, std=138.8, min=0, max=2036
  Test:  mean=52.6, std=58.2, min=0, max=187
  Train samples: 251, Test samples: 32

Bacolodcity:
  Train: mean=32.4, std=38.7, min=0, max=187
  Test:  mean=14.3, std=13.6, min=0, max=56
  Train samples: 251, Test samples: 32

Capiz:
  Train: mean=41.8, std=82.9, min=0, max=583
  Test:  mean=18.2, std=12.8, min=1, max=38
  Train samples: 251, Test samples: 32

Guimaras:
  Train: mean=12.7, std=34.7, min=0, max=276
  Test:  mean=6.5, std=6.7, min=0, max=22
  Train samples: 251, Test samples: 32

Iloilo:
  Train: mean=145.9, std=340.2, min=0, max=2429
  Test:  mean=54.4, std=49.8, min=2, max=162
  Train samples: 251, Test samples: 32

Iloilocity:
 

In [46]:
# Plot predictions for each location
fig, axes = plt.subplots(4, 2, figsize=(18, 16))
axes = axes.ravel()

for i, location in enumerate(location_names):
    ax = axes[i]
    
    # Calculate R2 and RMSE for subplot title
    r2 = r2_score(y_test[:, i], y_pred_test[:, i])
    rmse = np.sqrt(mean_squared_error(y_test[:, i], y_pred_test[:, i]))
    
    # Plot test data
    ax.plot(dates_test, y_test[:, i], label='Actual', linewidth=2, marker='o', markersize=3, alpha=0.7)
    ax.plot(dates_test, y_pred_test[:, i], label='Predicted', linewidth=2, marker='x', markersize=3, alpha=0.7)
    
    ax.set_title(f'{location} (R²={r2:.3f}, RMSE={rmse:.1f})', fontsize=12, fontweight='bold')
    ax.set_xlabel('Date', fontsize=10)
    ax.set_ylabel('Cases', fontsize=10)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(MODELS_DIR / 'predictions_v2.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Predictions plot saved")


✅ Predictions plot saved


In [ ]:
# Save final model
final_model_path = MODELS_DIR / 'dengue_lstm_v2.keras'
model.save(final_model_path)
print(f"✅ Final V2 model saved to: {final_model_path}")

# Save predictions
predictions_df = pd.DataFrame({
    'Date': pd.to_datetime(dates_test),
    **{f'{loc}_Actual': y_test[:, i] for i, loc in enumerate(location_names)},
    **{f'{loc}_Predicted': y_pred_test[:, i] for i, loc in enumerate(location_names)}
})
predictions_df.to_csv(MODELS_DIR / 'test_predictions_v2.csv', index=False)
print(f"✅ Test predictions saved to: {MODELS_DIR / 'test_predictions_v2.csv'}")

# Save training configuration
config = {
    'model_type': 'lstm_v2_simplified_for_small_dataset',
    'lookback': LOOKBACK,
    'n_features': X_train_scaled.shape[2],
    'n_original_features': len(feature_cols),
    'feature_reduction': f"{len(feature_cols)} → {X_train_scaled.shape[2]}",
    'n_targets': output_dim,
    'train_samples': len(X_train),
    'val_samples': len(X_val),
    'test_samples': len(X_test),
    'test_rmse': float(np.sqrt(test_mse)),
    'test_mae': float(test_mae),
    'avg_r2': float(metrics_df['R2'].mean()),
    'positive_r2_count': int((metrics_df['R2'] > 0).sum()),
    'good_r2_count': int((metrics_df['R2'] > 0.3).sum()),
    'location_names': location_names,
    'selected_features': important_features,
    'architecture': {
        'lstm_units': 32,
        'lstm_dropout': 0.4,
        'dense_layers': [32],
        'dense_dropout': [0.3],
        'l2_regularization': 0.001,
        'learning_rate': 0.001
    },
    'improvements_from_optimized': [
        'Minimized LSTM units (64 → 32) for small dataset',
        'Strong L2 regularization (0.0005 → 0.001)',
        'Stronger dropout: LSTM 0.4, Dense 0.3',
        'Simplified architecture: 1 LSTM + 1 Dense layer',
        'Aggressive feature selection (100 → ~50 features)',
        'Reduced lookback (6 → 4 weeks, dengue ~4-week lag)',
        'Maximized training split (70% → 80%)',
        'No early stopping - train all 200 epochs'
    ]
}

import json
config_file = MODELS_DIR / 'model_config_v2.json'
with open(config_file, 'w') as f:
    json.dump(config, f, indent=2, default=str)
print(f"✅ Model configuration saved to: {config_file}")

print("\n" + "="*70)
print("✅ V2 MODEL TRAINING COMPLETE!")
print("="*70)
print(f"\n📊 Final Results:")
print(f"   Test RMSE: {np.sqrt(test_mse):.2f} cases")
print(f"   Test MAE: {test_mae:.2f} cases")
print(f"   Average R²: {metrics_df['R2'].mean():.3f}")
print(f"   Locations with positive R²: {(metrics_df['R2'] > 0).sum()}/8")
print(f"   Locations with good R² (>0.3): {(metrics_df['R2'] > 0.3).sum()}/8")

print(f"\n💾 All artifacts saved to: {MODELS_DIR}")
print(f"\n📋 Next: Run diagnostic cell (12.5) to analyze performance patterns")

✅ Final V2 model saved to: c:\Users\miray\Desktop\dengue_forecasting_project\models\dengue_lstm_v2.keras
✅ Test predictions saved to: c:\Users\miray\Desktop\dengue_forecasting_project\models\test_predictions_v2.csv
✅ Model configuration saved to: c:\Users\miray\Desktop\dengue_forecasting_project\models\model_config_v2.json

✅ V2 MODEL TRAINING COMPLETE!

📊 Final Results:
   Test RMSE: 40.43 cases
   Test MAE: 22.18 cases
   Average R²: -0.455
   Locations with positive R²: 4/8
   Locations with good R² (>0.3): 2/8
